# SuAVE LeNet CNN Model Generator
For a simple Python implementation see https://www.pyimagesearch.com/2016/08/01/lenet-convolutional-neural-network-in-python/

The model was originally described in http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf

In [76]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [98]:
_ijt=qaqchumi3s6skmvg0s6gg5eerr%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

<IPython.core.display.Javascript object>

### Import all packages (this might take a few seconds)

In [78]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from imutils import paths

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras import backend as K
# import local lenet.py file describing the LeNet implementation with RELU activation functions
from lenet import LeNet

# More imports
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import csv
import pandas as pd
import re
import cv2
import os

# import the necessary packages for SVM predictor
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import imutils

# Import widget functionality
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Initializing the number of times the model will loop and its batch size for learning

In [79]:
epochCount = {
    '25 Iterations': 25,
    '50 Iterations': 50,
    '75 Iterations': 75
}

def f(epoch_count):
    return epoch_count

epochNum = interact(f, epoch_count=epochCount)

interactive(children=(Dropdown(description='epoch_count', options={'25 Iterations': 25, '50 Iterations': 50, '…

In [80]:
batchS = {
    '32 Batch Size': 32,
    '64 Batch Size': 64,
    '128 Batch Size': 128
}

def f(batch_size):
    return batch_size

batchNum = interact(f, batch_size=batchS)

interactive(children=(Dropdown(description='batch_size', options={'128 Batch Size': 128, '64 Batch Size': 64, …

In [81]:
imSize = {
    '20 by 20 Pixels': 20,
    '30 by 30 Pixels': 30,
    '50 by 50 Pixels': 50,
    '60 by 60 Pixels': 60,
    '80 by 80 Pixels': 80,
    '100 by 100 Pixels': 100,
    '120 by 120 Pixels': 120
}

print("Select image dimesions for resizing")

def f(img_dim):
    return img_dim

imDim = interact(f, img_dim=imSize)

Select image dimesions for resizing


interactive(children=(Dropdown(description='img_dim', options={'30 by 30 Pixels': 30, '50 by 50 Pixels': 50, '…

## Initialize variables

In [82]:
# init the number of epochs to train for, init learning rate and batch size
EPOCHS = epochNum.widget.result
INIT_LR = 1e-3
BS = batchNum.widget.result
# init the image suffix, yset, and image list
suffix = '.jpg'
img_list = []
yset = []
# create labels list and 2 dicts for 2 way mapping
labels = []
# key = label value = number
label_yval = dict()
# key = number value = label
yval_label = dict()
im_dimension = imDim.widget.result

## Run and choose the column name that coresponds with the column label to be predicted

In [83]:
# use csv file to grab images/labels
csv_path = "../../temp_csvs/" + csv_file
df = pd.read_csv(csv_path)

#generate image path
#lower_case_csv = csv_file.lower()
#lower_case_csv = lower_case_csv.split(user + "_")

dzc_file_array = dzc_file.split("/")
img_path = "../../images/" + dzc_file_array[-2] + "_" + dzc_file_array[-1].split(".")[0]



# Choose column of label for prediction
toPredict = list(df.columns.values)

pred_menu = {}
for i in range(0, len(toPredict)):
    pred_menu[toPredict[i]] = toPredict[i]

def f(predictions_menu):
    return predictions_menu
# choose which label for predictions
out2 = interact(f, predictions_menu=pred_menu)

interactive(children=(Dropdown(description='predictions_menu', options={'Taxonomic Name': 'Taxonomic Name', '#…

## Grab the images and configure them for predicting

### This might take a little while depending on the size of the dataset

In [84]:
im_dimension = 100
# grab chosen column names
nameCol = df['#img']
predCol = df[out2.widget.result]    
    
labels = []
# add all fabric columns to the y set
for i in range (0,len(predCol)):
    labels.append(predCol[i])

    
# grab all unique labels
uni_labels = set(labels)
uni_labels = list(uni_labels)

# assign each label a dict key number
for i in range(0,len(uni_labels)):
    yval_label[i] = uni_labels[i]
    label_yval[uni_labels[i]] = i


yset = []    
# create list of keys associated with their labels
for i in range (0, len(labels)):
    yset.append(label_yval[labels[i]])
    
img_list = []    
# gather images from path created from file names in csv file
for i in range (0,len(nameCol)):
    base_filename = nameCol[i]
    fileName = os.path.join(img_path, base_filename + suffix)
    im = cv2.imread(fileName)
    im = cv2.resize(im, (im_dimension,im_dimension))
    im = img_to_array(im)
    img_list.append(im)

# Shuffle the data
p = np.random.permutation(len(yset))


# Relable for splitting sets
Y = []
X = []
for i in range(0,len(yset)):
    Y.append(yset[p[i]])
    X.append(img_list[p[i]])
    
# split the test and training set 75:25
split = int(len(X)*(.75))

## Original DO NOT DELETE
xtrain = X[:split]
xtest = X[split:]
ytrain = Y[:split]
ytest = Y[split:]
    

# transform to arrays
trainX = np.array(xtrain, dtype="float")/255.0
testX = np.array(xtest, dtype ="float")/255.0

ytrain = np.array(ytrain)
ytest = np.array(ytest)

# parsed Y data containers
trainY = []
testY = []


# convert labels from int to vectors
trainY = np_utils.to_categorical(ytrain,len(uni_labels))
testY = np_utils.to_categorical(ytest,len(uni_labels))

# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                        horizontal_flip=True, fill_mode="nearest")
# initialize the model
model = LeNet.build(width=im_dimension, height=im_dimension, depth=3, classes=len(uni_labels))
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
                metrics=["accuracy"])

## Train the predictive model

### This is relative to the size of the data set and may take a few minutes 

In [85]:
# train the network
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, verbose=1)

Epoch 1/25
15/15 [==============================] - 56s 4s/step - loss: 1.5472 - acc: 0.3286 - val_loss: 1.0967 - val_acc: 0.4706
Epoch 2/25
15/15 [==============================] - 55s 4s/step - loss: 1.0960 - acc: 0.3975 - val_loss: 1.0945 - val_acc: 0.2941
Epoch 3/25
15/15 [==============================] - 56s 4s/step - loss: 1.0970 - acc: 0.3708 - val_loss: 1.0952 - val_acc: 0.3882
Epoch 4/25
15/15 [==============================] - 56s 4s/step - loss: 1.0980 - acc: 0.3441 - val_loss: 1.0949 - val_acc: 0.3882
Epoch 5/25
15/15 [==============================] - 57s 4s/step - loss: 1.0957 - acc: 0.3617 - val_loss: 1.0943 - val_acc: 0.3882
Epoch 6/25
15/15 [==============================] - 57s 4s/step - loss: 1.0997 - acc: 0.3417 - val_loss: 1.0941 - val_acc: 0.3882
Epoch 7/25
15/15 [==============================] - 58s 4s/step - loss: 1.0973 - acc: 0.3508 - val_loss: 1.0941 - val_acc: 0.3882
Epoch 8/25
15/15 [==============================] - 57s 4s/step - loss: 1.0975 - acc: 0.36

## Take the original data and predict based on the model

In [86]:
# Reshape original input data images for predicting
img_check = np.array(img_list, dtype ="float")/255.0

predictionsMade = []
preds = model.predict(img_check)

# Run all data through the prediction model that was created
for i in range (0,len(img_check)):
    predIndex = np.where(preds[i] == np.amax(preds[i]))
    prediction = int(predIndex[0][0])
    predictionsMade.append(prediction)
   
# Count how many correct predictions were made
correct = 0
for i in range (0,len(predictionsMade)):
    if(predictionsMade[i] == yset[i]):
        correct += 1 
        
print("Accuracy: " + str(correct/len(yset)))

Accuracy: 0.36323529411764705


## Generate model file and save

In [87]:
#Generate model file and save
modelName = user + "_cnn_" + out2.widget.result + "_" + str(epochNum.widget.result) + "_" + str(batchNum.widget.result) + ".h5"
modelPath = "models/"

model.save(os.path.join(modelPath, modelName))
print("Model file saved!")

Model file saved!


# Begin Testing on Data

In [88]:
#Load model
from keras.models import load_model
model2 = load_model(os.path.join(modelPath, modelName))

## Enter a new header for the prediction column

In [89]:
# Translate back to original csv label names
finalPred = []
for i in range (0,len(predictionsMade)):
    finalPred.append(yval_label[predictionsMade[i]])

from IPython.display import display
input_text = widgets.Text(
    value="pred " + out2.widget.result,
    placeholder='Type label here',
    disabled=False
)
output_text = widgets.Text(
    value="pred " + out2.widget.result,
    placeholder='New Header will be displayed here',
    disabled=False
)

def bind_input_to_output(sender):
    output_text.value = input_text.value

input_text.observe(bind_input_to_output)

print("Input new column Header Label: ")

display(input_text)
display(output_text)

Input new column Header Label: 


Text(value='pred Common Name', placeholder='Type label here')

Text(value='pred Common Name', placeholder='New Header will be displayed here')

## Write the predictions back to the original CSV

In [90]:
# Append the new column w/ it's new column name
df[input_text.value] = finalPred

#Get file path
path = "../../temp_csvs"

# new file name
new_file_0 = csv_file.split("_", 1)[1]
new_file =  new_file_0[:-4]+'_v1.csv'
new_file_path = os.path.join(path, new_file)
df.to_csv(os.path.join(path, new_file), index=None)

In [91]:
print(new_file)

test_restored_lennet_butteflies_v1.csv


## Display updated dataframe

In [92]:
pd.set_option('display.max_rows',1000)
df

,#img,#name,Title,Common Name,Taxonomic Name,new_50_64_100,predicted_common_name,pred Common Name
0,IMG_3238_1,IMG_3238_1,IMG_3238_1,Monarch,Danaus plexippus,Monarch,Monarch,Monarch
1,IMG_3238_2,IMG_3238_2,IMG_3238_2,Monarch,Danaus plexippus,Monarch,Monarch,Monarch
2,IMG_3238_3,IMG_3238_3,IMG_3238_3,Monarch,Danaus plexippus,Monarch,Monarch,Monarch
3,IMG_3238_4,IMG_3238_4,IMG_3238_4,Monarch,Danaus plexippus,Monarch,Monarch,Monarch
4,IMG_3238_5,IMG_3238_5,IMG_3238_5,Monarch,Danaus plexippus,Monarch,Monarch,Monarch
5,IMG_3238_6,IMG_3238_6,IMG_3238_6,Monarch,Danaus plexippus,Monarch,Monarch,Monarch
6,IMG_3239_1,IMG_3239_1,IMG_3239_1,Monarch,Danaus plexippus,Monarch,Monarch,Monarch
7,IMG_3239_2,IMG_3239_2,IMG_3239_2,Monarch,Danaus plexippus,Monarch,Monarch,Monarch
8,IMG_3239_3,IMG_3239_3,IMG_3239_3,Monarch,Danaus plexippus,Monarch,Monarch,Monarch
9,IMG_3239_4,IMG_3239_4,IMG_3239_4,Monarch,Danaus plexippus,Monarch,Monarch,Monarch


## Generate new survey name

In [93]:
#Input survey name

default_name = new_file.split(".")[0] + "_" + str(EPOCHS) + "_" + str(BS) + "_" + str(im_dimension)
default_name = default_name.replace("_", " ")


from IPython.display import display
input_text = widgets.Text(value=default_name)
output_text = widgets.Text(value=default_name)


def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.observe(bind_input_to_output)

print("Input survey name here:")
# Display input text box widget for input
display(input_text)

display(output_text)


Input survey name here:


Text(value='test restored lennet butteflies v1 25 32 100')

Text(value='test restored lennet butteflies v1 25 32 100')

## Parse URL for sending to SuAVE

In [101]:
survey_name = output_text.value
#Parse url
survey_url_0 = survey_url
survey_url_1 = survey_url
upload_url = survey_url_0.split("/main")[0]

if "https" in upload_url:
    upload_url = upload_url.replace("s","",1)
    upload_url = upload_url + ":3001"    

upload_url = upload_url + "/uploadCSV"
    
new_survey_url_base = survey_url_1.split(user)[0]

print(survey_url)

print("Uploading to: ", upload_url)

https://suave-dev.sdsc.edu/main/file=zaslavsk_test_restored_lennet_butteflies.csv
Uploading to:  http://suave-dev.sdsc.edu:3001/uploadCSV


## Send updated data back to SuAVE

In [102]:
import requests
upload_data = {'name': input_text.value, 'dzc': dzc_file, 'user':user}
files = {"file": open(new_file_path, "rb")}
r = requests.post(upload_url, files=files, data=upload_data)
print(r.status_code, r.reason)

regex = re.compile('[^0-9a-zA-Z_]')


url = new_survey_url_base + user + "_" + survey_name.replace(" ", "_") + ".csv" + "&views=" + views + "&view=" + view
print(url)
print ("Click the URL to open the new survey")


200 OK
https://suave-dev.sdsc.edu/main/file=zaslavsk_test_restored_lennet_butteflies_v8_25_32_100.csv&views=1110001&view=grid
Click the URL to open the new survey
